In [ ]:
# !pip install pyvi

In [59]:
import pandas as pd
from tqdm import tqdm
import numpy as np
import string
from pyvi import ViTokenizer, ViPosTagger
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score, classification_report, confusion_matrix
from tensorflow.keras import layers, models, callbacks, preprocessing, losses, optimizers, utils

In [144]:
# from google.colab import drive
# drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [145]:
# %cd /content/drive/My Drive/School/Năm 3/DataMining/FinalTerm

/content/drive/My Drive/School/Năm 3/DataMining/FinalTerm


In [146]:
# %ls

data/                model/                     text_classify_sequences_digit.pkl
Final_Project.gdoc   text-classification.ipynb  text_classify_tokenizer.pkl
generate_text.ipynb  text_classify_model.h5     text_generator.ipynb


## Import data from source

In [21]:
vnexpress = pd.read_csv('./data/vnexpress/csv/vnexpress.csv').drop(columns=['Unnamed: 0'], axis=1)
vnexpress.shape

(2481, 6)

In [22]:
vnexpress.head()

,article_id,content,topic,sub-topic,title,description
0,0,"""Xin lỗi đi"". Nhiều thế hệ đã dùng câu này để ...",doi-song,to-am,Có nên bắt trẻ xin lỗi?,Một số cha mẹ cho rằng việc ép trẻ nói lời xin...
1,1,Một nghiên cứu của đại học Wisconsin-Madison (...,doi-song,to-am,Tác dụng bất ngờ của việc gọi cho mẹ,"Khi bạn có một ngày tồi tệ như sếp mắng, mất c..."
2,2,"Các yếu tố như độ tuổi của trẻ, mối quan hệ gi...",doi-song,to-am,"Thiệt thòi của trẻ 'nửa tuần ở với bố, nửa tuầ...","Khi cha mẹ mỗi người ở một nhà sau ly hôn, việ..."
3,3,Ở tuổi đôi mươi chị Trần Thị Liên kết hôn với ...,doi-song,to-am,Bi kịch ở tuổi 50 của người phụ nữ Hậu Giang,"Trong nhà tình nghĩa, vợ chồng chị Liên, 50 tu..."
4,4,"Chị Hoàng Thị Hòa, 34 tuổi, ở Hà Nam lấy chồng...",doi-song,to-am,Sốc văn hóa nhà chồng,"Về làm dâu, chị Hòa nhận ra đã bước chân vào m..."


In [23]:
vnexpress['tag'] = vnexpress['content'] + vnexpress['title'] + vnexpress['description']
vnexpress.drop(columns=['content', 'title', 'description'], inplace=True)

In [24]:
mask = vnexpress['tag'].isna()
vnexpress[mask]

,article_id,topic,sub-topic,tag
110,110,doi-song,bai-hoc-song,NaN
136,136,doi-song,bai-hoc-song,NaN
215,215,doi-song,nha,NaN
337,337,doi-song,tieu-dung,NaN
407,407,du-lich,diem-den,NaN
...,...,...,...,...
2338,2338,the-thao,photo,NaN
2353,2353,the-thao,photo,NaN
2361,2361,the-thao,photo,NaN
2381,2381,the-thao,photo,NaN


In [25]:
vnexpress.dropna(axis=0, inplace=True)
vnexpress.reset_index(inplace=True)
vnexpress.drop(columns=['index'], axis=1, inplace=True)
vnexpress.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 2418 entries, 0 to 2417
Data columns (total 4 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   article_id  2418 non-null   int64 
 1   topic       2418 non-null   object
 2   sub-topic   2418 non-null   object
 3   tag         2418 non-null   object
dtypes: int64(1), object(3)
memory usage: 75.7+ KB


In [26]:
vnexpress['topic'].value_counts()

topic
giai-tri    614
the-thao    498
khoa-hoc    410
doi-song    361
giao-duc    355
du-lich     180
Name: count, dtype: int64

In [27]:
news = vnexpress.copy()
label_encoder = LabelEncoder()
news['topic'] = label_encoder.fit_transform(vnexpress['topic'])

news.head()

,article_id,topic,sub-topic,tag
0,0,0,to-am,"""Xin lỗi đi"". Nhiều thế hệ đã dùng câu này để ..."
1,1,0,to-am,Một nghiên cứu của đại học Wisconsin-Madison (...
2,2,0,to-am,"Các yếu tố như độ tuổi của trẻ, mối quan hệ gi..."
3,3,0,to-am,Ở tuổi đôi mươi chị Trần Thị Liên kết hôn với ...
4,4,0,to-am,"Chị Hoàng Thị Hòa, 34 tuổi, ở Hà Nam lấy chồng..."


In [28]:
X = news[['article_id', 'tag']]
y = news[['topic']]
news_train, news_test, label_train, label_test = train_test_split(X, y, test_size=0.2, random_state=123)

In [29]:
print("The length of news_train: ", len(news_train))
print("The length of news_test: ", len(news_test))

The length of news_train:  1934
The length of news_test:  484


In [30]:
print(label_train.value_counts())
print(label_test.value_counts())

topic
2        483
5        393
4        336
0        301
3        272
1        149
Name: count, dtype: int64
topic
2        131
5        105
3         83
4         74
0         60
1         31
Name: count, dtype: int64


## Initialize the input sequences

In [31]:
def word_separation(sentences):
    return [sen.split() for sen in sentences]

In [32]:
def create_input(tokenizer, sequences:list, max_length:int):
    sequence_digit = tokenizer.texts_to_sequences(sequences)
    input_sequences = np.array(preprocessing.sequence.pad_sequences(sequence_digit, maxlen=max_length, padding='pre'))
    return input_sequences

In [33]:
def clean_document(doc):
    doc = ViTokenizer.tokenize(doc)
    doc = doc.lower() # lower
    tokens = doc.split() # split into words
    table = str.maketrans('', '', string.punctuation.replace("_", "")) # remove all punctuations
    tokens = [w.translate(table) for w in tokens]
    tokens = [word for word in tokens if word]
    return tokens

In [34]:
papers = news_train['tag'].tolist()
print(len(papers))
papers[0:1]

1934


['Khi bị động vật săn mồi tóm gọn, rắn dice trên đảo Golem Grad, hòn đảo giữa hồ ở Bắc Macedonia, tự làm bẩn chính nó bằng hỗn hợp phân và hợp chất mùi hăng. Sau đó, chúng bất động, miệng há to, lưỡi thè ra. Cuối cùng, để hoàn thiện màn giả chết, một số con thậm chí thổ huyết. Việc sử dụng máu để giả chết của loài rắn này có thể là mấu chốt để dựng lên màn biểu diễn thuyết phục, theo nghiên cứu công bố hôm 8/5 trên tạp chí Biology Letters. Nhiều loài trong vương quốc động vật giả chết khi bị động vật săn mồi quấy rầy, bao gồm côn trùng, cá và bò sát. Ngay cả động vật có vú cũng làm vậy. Hành vi giả chết có hiệu quả lớn đi kèm với rủi ro cao, theo Vukasin Bjelica, nghiên cứu sinh ở Đại học Belgrade, tác giả của bài báo. Một số động vật săn mồi tỏ ra bối rối hay kinh hãi khi thấy loài vật khác đột nhiên bất động, đặc biệt nếu chúng còn bốc mùi và chảy máu. Các động vật khác có thể ngừng theo dõi chăm chú và thả lỏng móng vuốt, giúp mồi săn có thời gian thoát thân. Nhưng điều đó đòi hỏi c

In [35]:
sequences = []
for seq in tqdm(papers):
  tokens = clean_document(seq)
  line = ' '.join(tokens)
  sequences.append(line)

100%|██████████| 1934/1934 [00:27<00:00, 69.16it/s]


In [158]:
print('Length of sequences:', len(sequences))
sequences[2]

Length of sequences: 1337


'kati_morton nhà trị_liệu hôn_nhân và gia_đình ở santa_monica california mỹ gần đây trong một video tiktok với hơn 6 triệu lượt xem đã liệt_kê những dấu_hiệu của hội_chứng con_gái đầu_lòng gồm cảm_giác mãnh_liệt về trách_nhiệm gia_đình xu_hướng làm hài_lòng mọi người và dễ nổi_giận với những người em ruột cách đây hơn một năm trên mạng xã_hội x từng có một câu hỏi được nhiều người quan_tâm bạn có hạnh_phúc khi là con đầu và là con_gái rất nhiều người bình_luận dưới bài viết rằng họ thường_xuyên tự nhận thấy phải gánh_vác trách_nhiệm trong gia_đình cũng như biết cách nhường_nhịn để làm hài_lòng người khác trong số các anh_chị_em chị cả cũng có xu_hướng phải đối_mặt với những kỳ_vọng cao hơn về thành_tích hành_vi và trách_nhiệm mà cha_mẹ đặt ra thậm_chí họ cũng được giao những công_việc trong gia_đình mà các em nhỏ không bao_giờ được giao điều này cũng đặt con_gái đầu_vào vị_trí chuyên đóng vai kẻ xấu trong nhà khi bị các em mô_tả là hách_dịch hoặc độc_đoán không_những thế họ còn phải đố

In [159]:
input_sequences = word_separation(sequences)
print(input_sequences[0:2])

[['người', 'mẹ', '42', 'tuổi', 'ở', 'đà_nẵng', 'vội_vã', 'tra_hỏi', 'và', 'vỡ_lẽ_hóa', 'ra', 'cậu', 'bé', 'hiểu', 'lầm', 'ngọc_trung', 'con_trai', 'chị', 'bắt_đầu_vào', 'tuổi', 'dậy', 'thì', 'từng', 'vài', 'lần', 'xuất_hiện', 'tình_trạng', 'cương', 'cứng', 'phải', 'xả', 'vào', 'bồn_cầu', 'một', 'lần', 'chị', 'gái', 'hàng_xóm', 'sang', 'dùng', 'nhờ', 'nhà', 'vệ_sinh', 'hai', 'tháng', 'sau', 'người', 'này', 'khoe', 'với', 'chị', 'hằng', 'là', 'đã', 'có_thai', 'ngọc_trung', 'hốt_hoảng', 'lên', 'internet', 'tìm_hiểu', 'thấy', 'nói', 'xuất', 'ngoài', 'vẫn', 'có_thể', 'mang', 'thai', 'cậu', 'bé', 'đinh_ninh', 'mình', 'là', 'bố', 'của', 'đứa', 'bé', 'trong', 'bụng', 'chị', 'hàng_xóm', 'nên', 'thú_tội', 'với', 'mẹ', 'thở_phào', 'vì', 'câu_chuyện', 'không', 'nghiêm_trọng', 'như', 'mình', 'tưởng', 'nhưng', 'ngay', 'sau', 'đó', 'chị', 'bích_hằng', 'nhận', 'ra', 'con', 'đang', 'thiếu_hụt', 'kiến_thức', 'về', 'giới_tính', 'và', 'tình_dục', 'trầm_trọng', 'trong', 'lúc', 'giải_thích', 'cho', 'trung',

In [160]:
# filter several punctuations in 50803 sentences
tokenizer = preprocessing.text.Tokenizer(filters='!“"”#$%&()*+,-./:;<=>?@[\]^`{|}~ ', oov_token='<oov>')
# create vocabulary (corpus) for each word in 50803 sentences
tokenizer.fit_on_texts(input_sequences)

print(len(tokenizer.word_index))
tokenizer.word_index

28860


{'<oov>': 1,
 'và': 2,
 'của': 3,
 'trong': 4,
 'được': 5,
 'là': 6,
 'cho': 7,
 'có': 8,
 'các': 9,
 'người': 10,
 'với': 11,
 'năm': 12,
 'một': 13,
 'không': 14,
 'để': 15,
 'nhiều': 16,
 'khi': 17,
 'ở': 18,
 'những': 19,
 'từ': 20,
 'đến': 21,
 'về': 22,
 'hơn': 23,
 'theo': 24,
 'nhà': 25,
 'trên': 26,
 'ông': 27,
 'cô': 28,
 'vào': 29,
 'như': 30,
 'làm': 31,
 'đó': 32,
 'cũng': 33,
 'anh': 34,
 'tại': 35,
 'tôi': 36,
 'đã': 37,
 'sau': 38,
 'này': 39,
 'ngày': 40,
 'hai': 41,
 'nói': 42,
 'ra': 43,
 'biết': 44,
 'sẽ': 45,
 'còn': 46,
 'tuổi': 47,
 'đi': 48,
 'lại': 49,
 'con': 50,
 'phải': 51,
 'nhưng': 52,
 'có_thể': 53,
 'tháng': 54,
 'qua': 55,
 'cùng': 56,
 'mới': 57,
 'nên': 58,
 'trước': 59,
 'sự': 60,
 'việc': 61,
 'đồng': 62,
 'bạn': 63,
 'nhất': 64,
 'nước': 65,
 'chỉ': 66,
 'mỗi': 67,
 'cao': 68,
 'bà': 69,
 'phim': 70,
 'bị': 71,
 'việt_nam': 72,
 '4': 73,
 'giúp': 74,
 'vì': 75,
 'lên': 76,
 'lần': 77,
 'mình': 78,
 'lao_động': 79,
 'do': 80,
 'từng': 81,
 'đây': 82

In [36]:
# max_length = news['tag'].map(len).max()
max_length = max([len(i) for i in sequences])
max_length

19362

In [162]:
topic_size = len(news['topic'].unique()) + 1
topic_size

5

In [163]:
X_train = create_input(tokenizer, sequences, max_length)

print(len(X_train[0]))
print(X_train[0:2])

15867
[[    0     0     0 ...  7548  1038 17872]
 [    0     0     0 ...   107    40  1107]]


In [164]:
## Sửa định dạng y
y_train = np.array([label_train['topic']])
y_train = y_train.reshape(-1, 1)
y_train = utils.to_categorical(y_train, num_classes=topic_size)

print(y[0:5])

[[1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]
 [1. 0. 0. 0. 0.]]


## Design the LSTM model

In [165]:
vocab_size = len(tokenizer.word_index) + 1
vocab_size

28861

In [ ]:
drop_out = 0.2
output_unit = topic_size
embedding_size = 128

In [166]:
model = models.Sequential([
  layers.Embedding(vocab_size, embedding_size, input_length=max_length),
  layers.BatchNormalization(),
  layers.LSTM(64, return_sequences=True),
  layers.LSTM(64),
  layers.Dropout(drop_out),
  layers.Dense(9, activation='relu'),
  layers.Dense(9, activation='relu'),
  layers.Dense(9, activation='relu'),
  layers.Dense(9, activation='relu'),
  layers.Dense(unit=output_unit, activation='softmax')
])

model.summary()

Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_2 (Embedding)     (None, 15867, 128)        3694208   
                                                                 
 batch_normalization_2 (Bat  (None, 15867, 128)        512       
 chNormalization)                                                
                                                                 
 lstm_4 (LSTM)               (None, 15867, 64)         49408     
                                                                 
 lstm_5 (LSTM)               (None, 64)                33024     
                                                                 
 dropout_2 (Dropout)         (None, 64)                0         
                                                                 
 dense_10 (Dense)            (None, 9)                 585       
                                                      

In [167]:
model.compile(loss='categorical_crossentropy', optimizer='adam', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=25, batch_size=128)

## Dump model into files

In [177]:
# %cd /content/drive/My Drive/School/Năm 3/DataMining/FinalTerm/model

/content/drive/My Drive/School/Năm 3/DataMining/FinalTerm/model


In [178]:
import pickle
model.save('text_classify_model.h5')

with open('text_classify_tokenizer.pkl', 'wb') as f:
    pickle.dump(tokenizer, f)

/usr/local/lib/python3.10/dist-packages/keras/src/engine/training.py:3103: UserWarning: You are saving your model as an HDF5 file via `model.save()`. This file format is considered legacy. We recommend using instead the native Keras format, e.g. `model.save('my_model.keras')`.
  saving_api.save_model(


## Predict samples and Evaluate on test dataset

In [11]:
import pickle
model = models.load_model('./model/text_classify_model.h5')

with open('./model/text_classify_tokenizer.pkl', 'rb') as f:
    tokenizer = pickle.load(f)

In [38]:
def normalize_text(text, tokenizer, max_length):
    tokeninzed_sequences = clean_document(text)
    sequences_digit = tokenizer.texts_to_sequences([tokeninzed_sequences])
    sequences_digit_padding = np.array(preprocessing.sequence.pad_sequences(sequences_digit, maxlen=max_length, padding='pre'))
    return sequences_digit_padding

* Predict a sample

In [13]:
sample_text = '''
Khi bạn có một ngày tồi tệ như sếp mắng, mất chìa khóa hay thiếu tiền thanh toán hóa đơn khiến cuộc sống căng thẳng, các nhà khoa học khuyên nên gọi cho mẹ.

Một nghiên cứu của đại học Wisconsin-Madison (Mỹ) xem xét ba loại hình giao tiếp với mẹ như nói chuyện trực tiếp, qua điện thoại hoặc nhắn tin tức thời (chat) để xác định loại nào có lợi nhất. Họ cũng quan tâm đến việc có điều gì đặc biệt giá trị khi một ai đó nghe giọng nói của mẹ hay không.

Các nhà nghiên cứu yêu cầu 68 trẻ em từ 7,5 đến 12 tuổi trải qua một tình huống căng thẳng liên quan đến hoàn thành các bài toán và nói trước khán giả. Ví dụ trong khoảng thời gian rất ngắn, những người tham gia phải tính ra kết quả của phép toán như 7.648 x 9, đọc ngược dãy số đó trước một hội đồng và nếu mắc lỗi phải bắt đầu lại.
'''

In [40]:
temp = normalize_text(text=sample_text, tokenizer=tokenizer, max_length=15867)
print(temp)

prediction = model.predict(temp)
print(prediction)

[[  0   0   0 ...  51 410  49]]
1/1 [==============================] - 3s 3s/step
[[9.1036189e-01 1.1726055e-02 3.4974266e-02 4.2152558e-02 7.8524090e-04]]


In [41]:
result = label_encoder.inverse_transform([np.argmax(prediction[0])])
result

array(['doi-song'], dtype=object)

* Evaluate performance

In [42]:
papers = news_test['tag'].tolist()
print(len(papers))
papers[0:1]

484


['Jose Saldana đến Việt Nam năm 2019 để nối tiếp hành trình khám phá thế giới của mình. Bị điếc năm hai tuổi nhưng Jose luôn khát khao chinh phục những đỉnh núi cao nhất và chu du khắp nơi nên chọn nghề nhiếp ảnh. Anh đến Việt Nam trong lịch trình khám phá Đông Nam Á. Dịp ấy, Jose kết nối với những người bạn Mỹ đang sống ở Việt Nam. Họ mời thêm những người bạn điếc khác, trong đó có Bùi Lan Anh, 29 tuổi. Buổi đó, nhóm người trẻ chụm đầu vào nhau nói đủ chuyện bằng ngôn ngữ ký hiệu. Jose bị hút về phía Lan Anh, một cô gái xinh đẹp, hài hước và sôi nổi nhất. Anh giơ máy ảnh chụp hình cả nhóm và bật cười khi thấy Lan Anh làm trò trêu mình. "Khoảnh khắc đó, tôi biết chắc Lan Anh là một nửa hoàn hảo của đời mình", Jose kể. Phải lòng Lan Anh nhưng lúc đó Jose không biết cô từng là hoa khôi người điếc Việt Nam năm 2015 và á hậu 2 cuộc thi Miss and Mister Deaf International (Hoa hậu và Nam vương người điếc quốc tế) tại Mỹ năm 2016. Bùi Lan Anh quê Vĩnh Phúc, bị điếc bẩm sinh, tốt nghiệp trường

In [43]:
sequences_test = []
for seq in tqdm(papers):
  tokens = clean_document(seq)
  line = ' '.join(tokens)
  sequences_test.append(line)

100%|██████████| 484/484 [00:11<00:00, 41.40it/s]


In [44]:
X_test = create_input(tokenizer, sequences_test, 15867)

print(len(X_test[0]))
print(X_test[0:2])

15867
[[   0    0    0 ... 8344    3   34]
 [   0    0    0 ... 3362  297  101]]


In [52]:
prediction = model.predict(X_test)
print(prediction)

16/16 [==============================] - 86s 5s/step
[[8.9570934e-01 4.7643930e-03 6.7704871e-02 3.0775851e-02 1.0455964e-03]
 [3.9201319e-01 4.6249345e-01 5.8712950e-03 1.3809374e-01 1.5283183e-03]
 [8.7747192e-01 3.2715794e-02 2.2415362e-02 6.6460952e-02 9.3596004e-04]
 ...
 [7.9165041e-01 7.7098139e-02 2.3996063e-02 1.0520739e-01 2.0480028e-03]
 [1.7290922e-02 9.5110929e-01 3.1761595e-04 3.0854048e-02 4.2813487e-04]
 [1.4026935e-02 9.5876873e-01 2.3841855e-04 2.6636221e-02 3.2971709e-04]]


In [54]:
predictions = [np.argmax(pred) for pred in prediction]
print(mean_squared_error(label_test, predictions))
print(r2_score(label_test, predictions))

8.551652892561984
-2.25230901608072


In [60]:
print(confusion_matrix(label_test, predictions))

[[ 60   0   0   0   0   0]
 [ 21  10   0   0   0   0]
 [ 24 107   0   0   0   0]
 [ 57  26   0   0   0   0]
 [ 64  10   0   0   0   0]
 [ 56  49   0   0   0   0]]


In [58]:
print(classification_report(label_test, predictions))

              precision    recall  f1-score   support

           0       0.21      1.00      0.35        60
           1       0.05      0.32      0.09        31
           2       0.00      0.00      0.00       131
           3       0.00      0.00      0.00        83
           4       0.00      0.00      0.00        74
           5       0.00      0.00      0.00       105

    accuracy                           0.14       484
   macro avg       0.04      0.22      0.07       484
weighted avg       0.03      0.14      0.05       484



d:\Programming\Vietnamese-Text-Classification-and-Clustering\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Programming\Vietnamese-Text-Classification-and-Clustering\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control this behavior.
  _warn_prf(average, modifier, f"{metric.capitalize()} is", len(result))
d:\Programming\Vietnamese-Text-Classification-and-Clustering\.venv\Lib\site-packages\sklearn\metrics\_classification.py:1509: UndefinedMetricWarning: Precision is ill-defined and being set to 0.0 in labels with no predicted samples. Use `zero_division` parameter to control th